# A Study of Transcription and Its Affects
---
**Contributors:** John M. Letey (John.Letey@colorado.edu), David A. Knox (David.Knox@colorado.edu)

**Notes:** To get to my GitHub repository, go to this [website](https://github.com/JohnLetey/A-Study-of-Transcription-and-Its-Affects). For instructions on how to use my GitHub repository, go [here](https://github.com/JohnLetey/A-Study-of-Transcription-and-Its-Affects/blob/current/instructions.md).

**Dependencies:** My Jupyter Notebook uses the [biopython](https://github.com/biopython/biopython) package, which you can easily download using the built in interface in the Anaconda Navigator. Look at my instructions [here](https://github.com/JohnLetey/A-Study-of-Transcription-and-Its-Affects/blob/current/instructions.md) on how to download biopython. I also have the dependency of having the package [pandas](https://github.com/pandas-dev/pandas), which should come pre-installed when you download Anaconda.

This project follows the below pipeline.

![pipeline](../Pictures/pipeline.png?raw=true)

I have broken this project into to two major portions. The first portion calculates the hits (weak and strong transcription factor binding sights) for a specified chromosome (or all of the chromosomes) and transcription factor. The second portion analyses these hits. Before we talk about these two parts more in depth, I'm going to give you a brief introduction on why I'm doing this.

## Introduction
---
The fundamental component of life, whether we're talking about animals or humans or anything, is the cell. The cell is the most complex and unique part of any living being. As humans, some of us wonder what goes on inside such a fundamental component of us. That when I started wondering, is there a pattern? In all cells, the DNA goes through a process called transcription which converts it into RNA (which then the cell uses to make proteins). While doing this, there are transcription factors, which the whole point of there existence is to bind to the DNA. Each transcription factor has a unique target string which it is looking for in the DNA sequence. 

## Imports
---

In [1]:
import Bio
from Bio import SeqIO
import pandas as pd
from time import time
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os

## Part 1: Finding the Hits
---
To find the hits, we have to have the three components

1. Get the chromosomes from a specific fasta file (using the Python package biopython to read the file into a pandas dataframe).
2. Get the PSSMs for all possible transcription factors from a specific tamo file
3. Implement a function that calculates the output of a PSSM given a specific sequence

When I am done implementing this, I'll move on to finding the hits.

<!--<span style='color:blue'>some blue text</span>-->

Let's implement a function that gets all of the data from a specified fasta file and puts it into a pandas dataframe using the Python module biopython.

In [2]:
def getCHRs(filename, numUseless):
    # Get the data from the fasta file
    with open(filename) as fasta_file:
        identifiers = []
        seq = []
        for seq_record in SeqIO.parse(fasta_file, 'fasta'):
            identifiers.append(seq_record.id)
            seq.append(str(seq_record.seq))
    # Put the data into a dataframe
    s1 = pd.Series(identifiers[:len(identifiers)-numUseless], name='chromosome')
    s2 = pd.Series(seq[:len(seq)-numUseless], name='sequence')
    fastaDF = pd.DataFrame(dict(chromosome=s1, sequence=s2))
    # Return the dataframe
    return fastaDF

Now that I've defined the function that reads in the entire fasta file and puts it into a dataframe, let's call that function on my specific data, and also show what the dataframe looks like. I also need to take away some of the data that I'm not going to use, specifically, chrmt and 2-micron.

In [3]:
# Get the data from the fasta file that I'm working with and take away two of those
fastaDF = getCHRs('Data/SGDv3.fasta', 2)
# Get the amount of chromosomes
countCHR = fastaDF.count()
numOfCHR = countCHR['chromosome']
# Show the dataframe
fastaDF

,chromosome,sequence
0,chr1,CCACACCACACCCACACACCCACACACCACACCACACACCACACCA...
1,chr2,AAATAGCCCTCATGTACGTCTCCTCCAAGCCCTGTTGTCTCTTACC...
2,chr3,CCCACACACCACACCCACACCACACCCACACACCACACACACCACA...
3,chr4,ACACCACACCCACACCACACCCACACACACCACACCCACACACCAC...
4,chr5,CGTCTCCTCCAAGCCCTGTTGTCTCTTACCCGGATGTTCAACCAAA...
5,chr6,GATCTCGCAAGTGCATTCCTAGACTTAATTCATATCTGCTCCTCAA...
6,chr7,CCACACCCACACACACCACACCCACACCCACACACTACCCTAACAC...
7,chr8,CCCACACACACCACACCCACACACCACACCCACACTTTTCACATCT...
8,chr9,CACACACACCACACCCACACCACACCACACCACACCCACACCCACA...
9,chr10,CCCACACACACACCACACCCACACCCACACACACCACACCCACACA...


I've read in all of the data I'm going to use from a specific fasta file and put it into a dataframe. Now I need to read in the all of the PSSMs from a specified tamo file into a dataframe.

In [4]:
def getPSSMs(filename):
    # Get the PSSMs from the tamo file
    tamo_file = open(filename)
    tamo_data = []
    line = tamo_file.readline()
    while line:
        tamo_data.append(line[:len(line)-1])
        line = tamo_file.readline()
    TF = []
    PSSM = []
    revPSSM = []
    for i in range(len(tamo_data)):
        if tamo_data[i][:6] == 'Source':
            pssm = pd.DataFrame(pd.Series(['A', 'C', 'T', 'G'], name='Letter'))
            REVpssm = pd.DataFrame(pd.Series(['A', 'C', 'T', 'G'], name='Letter'))
            revpssm = []
            lenOfPSSM = int((len(tamo_data[i-16]) - 3)/10) - 1
            firstLine = tamo_data[i-16]
            secondLine = tamo_data[i-15]
            thirdLine = tamo_data[i-14]
            fourthLine = tamo_data[i-13]
            for j in range(lenOfPSSM):
                value1 = float(firstLine[(j*10 + 6):(j*10 + 12)])
                value2 = float(secondLine[(j*10 + 6):(j*10 + 12)])
                value3 = float(thirdLine[(j*10 + 6):(j*10 + 12)])
                value4 = float(fourthLine[(j*10 + 6):(j*10 + 12)])
                revpssm.append([value1, value2, value3, value4])
                pssm.insert(j+1, str(j), pd.Series([value1, value2, value3, value4]))
            temp = revpssm
            revpssm = []
            for j in range(4):
                List = []
                for k in range(len(temp)):
                    List.append(temp[k][j])
                revpssm.append(List)
            for j in range(4):
                revpssm[j] = revpssm[j][::-1]
            temp = revpssm[0]
            revpssm[0] = revpssm[2]
            revpssm[2] = temp
            temp = revpssm[1]
            revpssm[1] = revpssm[3]
            revpssm[3] = temp
            temp = revpssm
            revpssm = []
            for j in range(len(temp[0])):
                List = []
                for k in range(4):
                    List.append(temp[k][j])
                revpssm.append(List)
            for j in range(lenOfPSSM):
                REVpssm.insert(j+1, str(j), pd.Series(revpssm[j]))
            TF.append(tamo_data[i][9:])
            PSSM.append(pssm)
            revPSSM.append(REVpssm)
    # Put the data into a dataframe
    s1 = pd.Series(TF, name='TF')
    s2 = pd.Series(PSSM, name='ForwardsPSSM')
    s3 = pd.Series(revPSSM, name='ReversePSSM')
    pssms = pd.DataFrame(dict(TF=s1, ForwardsPSSM=s2, ReversePSSM=s3))
    # Return the dataframe
    return pssms

Let's get the dataframe that contains all of the PSSMs for each transcription factor from my specific tamo file, and also show what the dataframe looks like. To access any one PSSM and it's reverse compliment, look up the corresponding index of the transcription factor you want (let's call that number `value`), them plug that value into the Python statement
```python
tamoDF['ForwardsPSSM'][value] # Get the forwards PSSM
tamoDF['ReversePSSM'][value]  # Get the reverse compliment PSSM
```
If you execute the above line, it should give you your PSSM for the corresponding transcription factor you wanted.

In [5]:
# Get the data from the tamo file that I'm working with
tamoDF = getPSSMs('Data/yeast.tamo')
# Get the amount of PSSMs (or TFs, both are the same)
countTF = tamoDF.count()
numOfTF = countTF['TF']
# Show the dataframe
tamoDF

,ForwardsPSSM,ReversePSSM,TF
0,Letter 0 1 2 3 4 ...,Letter 0 1 2 3 4 ...,HSF1
1,Letter 0 1 2 3 4 ...,Letter 0 1 2 3 4 ...,RPN4
2,Letter 0 1 2 3 ...,Letter 0 1 2 3 ...,THI2
3,Letter 0 1 2 3 4 5 ...,Letter 0 1 2 3 4 5 ...,UGA3
4,Letter 0 1 2 3 4 ...,Letter 0 1 2 3 4 ...,SMP1
5,Letter 0 1 2 3 ...,Letter 0 1 2 3 ...,RTG3
6,Letter 0 1 2 3 4 ...,Letter 0 1 2 3 4 ...,CIN5
7,Letter 0 1 2 3 4 ...,Letter 0 1 2 3 4 ...,DAL82
8,Letter 0 1 2 3 0 A ...,Letter 0 1 2 3 0 A ...,DAL80
9,Letter 0 1 2 3 4 ...,Letter 0 1 2 3 4 ...,DAL81


We have just read in all of the PSSMs for each specific transcription factor. There is one more component that we need to implement before we're all set to go ahead and calculate all of the hits. That component is a function that takes in a sequence (that is the same length (horizontal length) as the PSSM) and a PSSM and calculates the output of the PSSM when you plug in that sequence. Let's go ahead and do that.

In [6]:
def outputOfPSSM(PSSM, sequence):
    # Find the length (horizontal length) of the PSSM
    lenOfPSSM = PSSM.count(1) - 1
    lenOfPSSM = lenOfPSSM[0]
    # Find the maximum possible output of the PSSM
    maximumList = list(PSSM.max())
    del maximumList[0]
    maximum = 0
    for i in range(len(maximumList)):
        maximum += maximumList[i]
    # Calculate the output of the PSSM
    output = 0
    for i in range(len(sequence)):
        if sequence[i] == 'A':
            output += PSSM[str(i)][0]
        if sequence[i] == 'C':
            output += PSSM[str(i)][1]
        if sequence[i] == 'T':
            output += PSSM[str(i)][2]
        if sequence[i] == 'G':
            output += PSSM[str(i)][3]
    output /= maximum
    # Return the output of the PSSM
    return output

Now that we have all of the components, let's calculate the hits.

In [ ]:
def calculateHits(fastaDF, tamoDF, numOfTF, numOfCHR, weakThreshold, strongThreshold, Path):
    # Get the time
    t_beg = time()
    # Check to see if the path is made (if not, make it)
    if not os.path.exists(Path):
        os.makedirs(Path)
    # Calculate hits
    for i in range(numOfTF):
        pssm = tamoDF['ForwardsPSSM'][i]
        REVpssm = tamoDF['ReversePSSM'][i]
        TF = tamoDF['TF'][i]
        lenOfPSSM = pssm.count(1) - 1
        lenOfPSSM = lenOfPSSM[0]
        filename = Path + '/Hits' + TF + '.gff'
        fileID = open(filename, 'w')
        for j in range(numOfCHR):
            CHR = fastaDF['sequence'][j]
            print('Calculating the hits for the transcription factor', TF, 'using chromosome', fastaDF['chromosome'][j])
            t_intermediate_beg = time()
            for k in range(len(CHR)):
                if (k + lenOfPSSM) > len(CHR):
                    output1 = outputOfPSSM(pssm, CHR[k:] + str([' ' for l in range(k + lenOfPSSM - 1 - len(CHR))]))
                    output2 = outputOfPSSM(REVpssm, CHR[k:] + str([' ' for l in range(k + lenOfPSSM - 1 - len(CHR))]))
                    end = len(CHR)
                else:
                    output1 = outputOfPSSM(pssm, CHR[k:(k + lenOfPSSM)])
                    output2 = outputOfPSSM(REVpssm, CHR[k:(k + lenOfPSSM)])
                    end = k + lenOfPSSM
                if output1 >= weakThreshold:
                    if output1 < strongThreshold:
                        fileID.write(fastaDF['chromosome'][j] + '\t' + TF + '\t' + 'hit' + '\t' + str(k) + '\t' + str(end) + '\t' + str(output1) + '\t' + '+' + '\t' + '.' + '\t' + 'weak' + '\n')
                    else:
                        fileID.write(fastaDF['chromosome'][j] + '\t' + TF + '\t' + 'hit' + '\t' + str(k) + '\t' + str(end) + '\t' + str(output1) + '\t' + '+' + '\t' + '.' + '\t' + 'strong' + '\n')
                if output2 >= weakThreshold:
                    if output2 < strongThreshold:
                        fileID.write(fastaDF['chromosome'][j] + '\t' + TF + '\t' + 'hit' + '\t' + str(k) + '\t' + str(end) + '\t' + str(output2) + '\t' + '-' + '\t' + '.' + '\t' + 'weak' + '\n')
                    else:
                        fileID.write(fastaDF['chromosome'][j] + '\t' + TF + '\t' + 'hit' + '\t' + str(k) + '\t' + str(end) + '\t' + str(output2) + '\t' + '-' + '\t' + '.' + '\t' + 'strong' + '\n')
            t_intermediate_end = time()
            print('It took', (t_intermediate_end - t_intermediate_beg)/60, 'minutes to calculate the hits for transcription factor', TF, 'using chromosome', fastaDF['chromosome'][j])
        fileID.close()
    # Get the time
    t_end = time()
    # Return the time it took to calculate the hits
    return t_end - t_beg

Let's talk about the reasoning behind this. I'm given the two dataframes that contain the fasta and the tamo data. I want to go through each trancription factor and calculate the hits for every single chromosome.

In [ ]:
time_est = calculateHits(fastaDF, tamoDF, numOfTF, numOfCHR, 0.35, 0.7, 'Hits Files')
time_est

Calculating the hits for the transcription factor HSF1 using chromosome chr1
It took 18.438492600123087 minutes to calculate the hits for transcription factor HSF1 using chromosome chr1
Calculating the hits for the transcription factor HSF1 using chromosome chr2
It took 117.31964531342189 minutes to calculate the hits for transcription factor HSF1 using chromosome chr2
Calculating the hits for the transcription factor HSF1 using chromosome chr3
It took 30.447948948542276 minutes to calculate the hits for transcription factor HSF1 using chromosome chr3
Calculating the hits for the transcription factor HSF1 using chromosome chr4
It took 168.77411204973856 minutes to calculate the hits for transcription factor HSF1 using chromosome chr4
Calculating the hits for the transcription factor HSF1 using chromosome chr5
It took 50.9314360499382 minutes to calculate the hits for transcription factor HSF1 using chromosome chr5
Calculating the hits for the transcription factor HSF1 using chromosome 

## Part 2: Analysing the Hits
---
We have already implemented a function to calculate the hits, given a transcription factor and a chromosome (or all of the chromosomes), we have to analyse these hits. I analyse the hits by 

In [ ]:
def hist(InFilename, HitsInFilename, PicFile):
    # Get the time at the begining of the function
    t_beg = time()
    # Open the input file that holds the hits
    fileID = open(HitsInFilename, 'r')
    # Initialize the list that will hold the input
    data = []
    # Read in the input file
    line = fileID.readline()
    while line:
        data.append(line[:len(line)-1])
        line = fileID.readline()
    # Parse the input
    weakSites = []
    strongSites = []
    for i in range(len(data)):
        line = data[i]
        if len(line) != 0:
            if line[0] != '#':
                # Get the position of the hit
                position = line[2:8]
                position = int(position)
                # Get the type of the hit
                typeOfHit = line[len(line)-8:len(line)-2]
                # Input the position into the correct list
                if typeOfHit == 'strong':
                    strongSites.append(position)
                if typeOfHit == '  weak':
                    weakSites.append(position)
    # Take out all of the duplicates in the lists holding the hits
#     newList = []
#     for i in range(len(weakSites)):
#         target = weakSites[i]
#         found = False
#         i = 0
#         while i < len(newList) and found == False:
#             if target == newList[i]:
#                 found = True
#             else:
#                 i += 1
#         if found == False:
#             newList.append(target)
#     weakSites = newList
#     newList = []
#     for i in range(len(strongSites)):
#         target = strongSites[i]
#         found = False
#         i = 0
#         while i < len(newList) and found == False:
#             if target == newList[i]:
#                 found = True
#             else:
#                 i += 1
#         if found == False:
#             newList.append(target)
#     strongSites = newList
    #weakSites = list(set(weakSites))
    #weakSites.sort()
    #strongSites = list(set(strongSites))
    #strongSites.sort()
    # Close the input file that holds the hits
    fileID.close()
    # Read in the input file
    Input = readInInput(InFilename)
    # Get the number of plus or minus range of what we're going to be anaylsing and the size of the buckets
    r = Input[5]
    s = Input[4]
    # Analyse the hits
    categories = [0 for i in range(r//s)]
#     for i in range(len(strongSites)):
#         for j in range(len(weakSites)):
#             value = abs(strongSites[i] - weakSites[j])
#             for k in range(r//s):
#                 if (value <= (s*k) and (value > s*(k-1))):
#                     categories[k] += 1
    for i in range(len(strongSites)):
        position1 = 0
        position2 = 0
        for j in range(len(weakSites)):
            if weakSites[j] <= strongSites[i]-r:
                position1 = j
            if weakSites[j] <= strongSites[i]+r:
                position2 = j
        for j in range(position1, position2):
            value = abs(strongSites[i] - weakSites[j])
            for k in range(r//s):
                if (value <= (s*k)) and (value > s*(k-1)):
                    categories[k] += 1
    # Plot the histogram
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
    ax.bar(range(len(categories)), categories, color='black', label='Histogram')
    plt.plot(range(len(categories)), [mean(categories) for i in range(len(categories))], color='red', label='Mean')
    plt.plot(range(len(categories)), [2*std(categories)+mean(categories) for i in range(len(categories))], color='steelblue', label='Standard Deviation (+2)')
    plt.plot(range(len(categories)), [std(categories)+mean(categories) for i in range(len(categories))], color='cyan', label='Standard Deviation (+1)')
    if mean(categories)-std(categories) >= 0:
        plt.plot(range(len(categories)), [mean(categories)-std(categories) for i in range(len(categories))], color='cyan', label='Standard Deviation (-1)')
    if mean(categories)-2*std(categories) >= 0:
        plt.plot(range(len(categories)), [mean(categories)-2*std(categories) for i in range(len(categories))], color='steelblue', label='Standard Deviation (-2)')
    ax.legend()
    path = ''
    for i in range(len(PicFile)):
        if PicFile[i] != '/':
            path += PicFile[i]
        else:
            break
    if not os.path.exists(path):
        os.makedirs(path)
    fig.savefig(PicFile)
    plt.show()
    # Get the time at the end of the function
    t_end = time()
    # Return the amount of seconds it took the function to run (in seconds)
    return t_end - t_beg

Now that we have 

In [ ]:
# Import the function that gets the time
from time import time

def main(InFilename):
    # Get the time at the begining of the function
    t_beg = time()
    
    Input = readInInput(InFilename)
    fileID = open(Input[1])
    line = fileID.readline()
    while line:
        if line[:6] == 'Source':
            print('Transcription Factor:', line[9:len(line)-1])
            time_est = calculateHits(Input[0], 'all', Input[1], line[9:len(line)-1], 'Hits/hits' + line[9:len(line)-1] + '.txt')
            print('It took', time_est, 'seconds to calculate the hits')
            time_est = hist(InFilename, 'Hits/hits' + line[9:len(line)-1] + '.txt', 'Histograms/histogram' + line[9:len(line)-1] + '.png')
            print('It took', time_est, 'seconds to analyse the hits')
        line = fileID.readline()
    fileID.close()  
    # Get the time at the end of the function
    t_end = time()
    # Return the amount of seconds it took the function to run (in seconds)
    return t_end - t_beg

main('Data/input.txt')